# Introduction
A look at KE CHP performance based on HH registration (site size) quintiles. 

In [40]:
## Setup
# load libs/modules
%run "../utils/load_envz.ipynb"

# Load data set 
DATA_DIR = "../dataset"
DATA_FILE = "all_KE_CHVs_data_1493.csv" 
dataset = pd.read_csv( "%s/%s" % (DATA_DIR, DATA_FILE) )

dataset.shape
dataset.dtypes


(1493, 21)

CHP ID                                object
Pregnancy                              int64
PNC Visit Rate                       float64
PNC Visit Missed - #                   int64
Total U1 Assessments                   int64
Total U1 Treatments                    int64
Total U5 Assessments                   int64
Total U5 Treatments                    int64
U5 Malaria                             int64
U5 Diarrhea                            int64
U5 Pneumonia                           int64
On Time Referral f/u Treatment U5    float64
Missed Visits                          int64
%mRDT                                float64
All Malaria Assessments                int64
All Malaria Treatments                 int64
Act 6pack Issue                        int64
mRDT Used                              int64
Sales                                float64
Community Events                       int64
Households Registered                  int64
dtype: object

In [56]:
## Functions 
def sampleStructure(dtset, grp):
    argz = { 
        "CHP ID": {"# CHPs": "count"},
        "Households Registered": { "# HHs": "sum", "Avg.CHP.Size": "mean"},
        "Pregnancy" : { "# Pregoz Cases": "sum"},
        "All Malaria Assessments" : { "# Malaria Cases": "sum"},
        "Total U5 Assessments" : { "# Child Cases": "sum"},
        "Total U1 Assessments" : { "# U1 Cases": "sum"}
    }
    return dtset.groupby(grp).agg( argz).round(0)


def summarize(dtset): 
    return dtset.describe().round(2)

def distributionz(dtset, colz):
    fig, ax = plt.subplots( figsize=(10,10))
    return dtset.hist( colz, ax=ax)

def boxplotz(dtset, colz, grp):
    fig, ax = plt.subplots( figsize=(10,10))
    return dtset.boxplot( colz, grp, ax)

def corrz(dtset, colz):
    return dtset[colz].corr().round(3)
    
def plotCorrz( cor_outz ):
    # Generate a mask for the upper triangle
    mask = np.zeros_like(cor_outz, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    return cor_outz, sns.heatmap(cor_outz, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

def quintilize(dtset, col, nbGrpz=5, labelzPrefix="Quint"):
    labelz = []
    for i in range(nbGrpz):
        labelz.append( labelzPrefix+"_"+str(i+1))
    dtset["Quints_"+col] = pd.qcut( dtset[col], nbGrpz, labels=labelz, duplicates='drop') 

In [57]:

## themes | variable - outcome mapping
argz_Units = ["CHP ID", "Households Registered"] 
argz_Ops = [ "Sales", "Community Events"]
argz_pregoz = [ "Pregnancy", "PNC Visit Rate", "PNC Visit Missed - #"]
argz_U1_care = ["Total U1 Assessments", "Total U1 Treatments"]
argz_malaria = [ "All Malaria Assessments", "All Malaria Treatments", "Act 6pack Issue", "mRDT Used", "%mRDT"]
argz_U5_diseases = ["U5 Malaria", "U5 Diarrhea", "U5 Pneumonia"]
argz_U5_care = ["Total U5 Assessments", "Total U5 Treatments", "On Time Referral f/u Treatment U5", "Missed Visits"] 

argz_EPMs = ["Households Registered", "Sales", "Community Events", "Total U5 Assessments", "Total U1 Assessments", "Pregnancy"]

## Additional Columns
dataset["Country"] = "KE"
#dataset["HHQuintiles"] = pd.qcut( dataset["Households Registered"], 5, labels=["HH_Quint_1", "HH_Quint_2", "HH_Quint_3", "HH_Quint_4", "HH_Quint_5"])
dnul = pd.Series( ["Total U5 Assessments"]).apply( lambda x: quintilize(dataset, x))

dataset["Pct_Assessments_U1_in_U5"] = dataset["Total U1 Assessments"] / dataset["Total U5 Assessments"]
dataset["Pct_Treatments_U1_in_U5"] = dataset["Total U1 Treatments"] / dataset["Total U5 Treatments"] 

dataset.dtypes 

ValueError: Bin labels must be one fewer than the number of bin edges

In [44]:
sampleStructure(dataset, "Country")

C:\Users\BILHA\Anaconda3\lib\site-packages\pandas\core\groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


CHP ID Households Registered                   Pregnancy  \
        # CHPs                 # HHs Avg.CHP.Size # Pregoz Cases   
Country                                                            
KE        1493                147956         99.0            408   

        All Malaria Assessments Total U5 Assessments Total U1 Assessments  
                # Malaria Cases        # Child Cases           # U1 Cases  
Country                                                                    
KE                         2986                 5393                 1216

In [31]:
for i in range(5):
    print(i)

0
1
2
3
4
